In [1]:
# !pip install findspark

In [2]:
# !pip install seaborn

In [3]:
# import findspark
# findspark.init('/home/henry/spark-3.3.0-bin-hadoop3/')

In [4]:
!pip install pyspark

In [5]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
from pyspark.sql import SparkSession
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [6]:
sc = SparkContext()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/10 03:01:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)

/home/henry/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [8]:
socket_stream = ssc.socketTextStream("127.0.0.1", 5555)

In [9]:
lines = socket_stream.window(20)

In [10]:
from collections import namedtuple
fields = ("tag", "count")
Comment = namedtuple('Comment', fields)

In [11]:
ssc.start()

22/07/10 03:01:36 ERROR StreamingContext: Error starting the context, marking it as stopped
java.lang.IllegalArgumentException: requirement failed: No output operations registered, so nothing to execute
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.streaming.DStreamGraph.validate(DStreamGraph.scala:169)
	at org.apache.spark.streaming.StreamingContext.validate(StreamingContext.scala:518)
	at org.apache.spark.streaming.StreamingContext.liftedTree1$1(StreamingContext.scala:578)
	at org.apache.spark.streaming.StreamingContext.start(StreamingContext.scala:577)
	at org.apache.spark.streaming.api.java.JavaStreamingContext.start(JavaStreamingContext.scala:557)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvok

IllegalArgumentException: requirement failed: No output operations registered, so nothing to execute

In [ ]:
time.sleep(60)
# Use Parenthesis for multiple lines or use \.
( lines.flatMap( lambda text: text.split( " " ) ) #Splits to a list
  .filter( lambda word: 'bphone' in word.lower() ) # Checks for hashtag calls
  .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word
  .reduceByKey( lambda a, b: a + b ) # Reduces
  .map( lambda rec: Comment( rec[0], rec[1] ) ) # Stores in a Tweet Object
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  .limit(10).registerTempTable("Comments") ) ) # Registers to a table.

In [ ]:
# ssc.stop()

In [ ]:
count = 0
while count < 10:
    time.sleep(120)
    try:
        top_10_tweets = sqlContext.sql( 'Select tag, count from Comments' )
        top_10_df = top_10_tweets.toPandas()
        display.clear_output(wait=True)
        plt.figure( figsize = ( 10, 8 ) )
        sns.barplot( x="count", y="tag", data=top_10_df)
        plt.show()
    except:
        time.sleep(5)
        print('Waiting')
    count = count + 1